In [1]:
import pandas as pd
import geonamescache  as gcn
import re as r

In [2]:
headline = open("headlines.txt","r")

In [3]:
f_array = []
fw_array = []
j= []
p = []
clean_array = []
y =0

with open('headlines.txt') as my_file:
    f_array = my_file.read().split('\n')
    my_file.close() 

for i in f_array:
    j = i
    x = j.split()
    fw_array.append(x)

clean_array = fw_array

for k in range(len(fw_array)):
    for l in range(len(fw_array[k])):
        #print(fw_array[k][l])
        az = r.sub(r'[^a-zA-Z ]', '',fw_array[k][l])
        clean_array[k][l] = az

#print(clean_array)


final_array = f_array

for k in range(len(f_array)):
        #print(fw_array[k][l])
        az = r.sub(r'[^a-zA-Z ]', '',f_array[k])
        final_array[k] = az
        
#print(final_array)



In [4]:
import csv
import pandas as pd
from pandas import DataFrame
import geonamescache
gc = geonamescache.GeonamesCache()

data = {'headline':[], 'countries':[],'cities':[] }
countries = [e for e in gc.get_countries_by_names().keys()]

def runmycode(a, b):
    for o in range(a, b): 
        countryfound = False
        cityfound = False
        poss_cty = []
        poss_ctry = []
        cty = ""
        ctry = "" 
        for s in range(len(clean_array[o])):       
#COUNTRIES LOGIC        
            if (clean_array[o][s] in countries) :
                ci = clean_array[o][s]
                poss_ctry.append(ci)
                if (not cityfound):
                    ctry1 = clean_array[o][s]
                    while (s < len(clean_array[o])-2):     
                        s = s + 1
                        ctry1 = ctry1 + " " + clean_array[o][s]
                        if (gc.get_cities_by_name(cty1) != [])  :
                            ci = ctry1
                            poss_cty.append(ci)
                            ctry1 = clean_array[o][s]
                            
            else:
                ctry = clean_array[o][s]
                while (s <= len(clean_array[o])-1):     
                    if (ctry in countries)  :
                        ci = ctry
                        poss_ctry.append(ci)
                        s = s + 1
                    else:
                        if (s <= len(clean_array[o])-2):
                            ctry = ctry + " " + clean_array[o][s+1]
                        s = s + 1        
#CITIES LOGIC
        cityfound = False
        poss_cty = []
        cty = ""
        for s in range(len(clean_array[o])): 
            if ((gc.get_cities_by_name(clean_array[o][s])) != []) :
                ci = clean_array[o][s]
                poss_cty.append(ci)
                if (not cityfound):
                    cty1 = clean_array[o][s]
                    if s < (len(clean_array[o]) - 1):
                        while (s < len(clean_array[o]) - 2):     
                            s = s + 1
                            cty1 = cty1 + " " + clean_array[o][s]
                            if (gc.get_cities_by_name(cty1) != []):
                                ci = cty1
                                poss_cty.append(ci)
                                cty1 = clean_array[o][s]

            else:
                cty = clean_array[o][s]
                while (s <= len(clean_array[o])-1):     
                    if ((gc.get_cities_by_name(cty)) != [])  :
                        ci = cty
                        poss_cty.append(ci)
                        s = s + 1
                    else:
                        if (s <= len(clean_array[o])-2):
                            cty = cty + " " + clean_array[o][s+1]
                        s = s + 1                           
                    
#determining final city                    
        if (len(poss_cty) !=0):
            x = len(poss_cty)
            initial = 0
            final = 0
            final_j = 0
            j = 0
            while j < x:
                initial = len(str(poss_cty[j]))
                if initial > final:
                    final = initial
                    final_j = j
                j = j + 1
            data['cities'].append(poss_cty[final_j])
#determining final country         
        if (len(poss_ctry) !=0):
            x = len(poss_ctry)
            initial = 0
            final = 0
            final_j = 0
            j = 0
            while j < x:
                initial = len(str(poss_ctry[j]))
                if initial > final:
                    final = initial
                    final_j = j
                j = j + 1
            data['countries'].append(poss_ctry[final_j])
                
### add to data dictionary                
        data['headline'].append(final_array[o])
        if (len(data['headline']) != len(data['cities'])):
            data['cities'].append('NaN')
        if (len(data['headline']) != len(data['countries'])):
            data['countries'].append('NaN')

runmycode(0,650)

#publish as dataframe

df = pd.DataFrame.from_dict(data)
#print(df)
df.head(10)

,headline,countries,cities
0,Zika Outbreak Hits Miami,NaN,Miami
1,Could Zika Reach New York City,NaN,New York City
2,First Case of Zika in Miami Beach,NaN,Miami
3,Mystery Virus Spreads in Recife Brazil,Brazil,Recife
4,Dallas man comes down with case of Zika,NaN,Dallas
5,Trinidad confirms first Zika case,NaN,Trinidad
6,Zika Concerns are Spreading in Houston,NaN,Houston
7,Geneve Scientists Battle to Find Cure,NaN,NaN
8,The CDC in Atlanta is Growing Worried,NaN,Atlanta
9,Zika Infested Monkeys in Sao Paulo,NaN,NaN


In [72]:
df_lat = []
df_long = []
df_ctrycd = []


#a_list - main list of geonamescache    
#a_dict - first dictionary


for i in df.index:
    if df['cities'][i] == 'NaN' or gc.get_cities_by_name(df['cities'][i]) == []:
        df_lat.append('NaN')
        df_long.append('NaN')
        df_ctrycd.append('NaN')
    else:
        a_list = gc.get_cities_by_name(df['cities'][i])
        for word in a_list:
            a_dict = word
            if len(a_list) == 1:
                for key, value in a_dict.items():
                    x_lat_f = value['latitude']
                    x_long_f= value['longitude']
                    x_ctrycd_f = value['countrycode']
            else:
                j = 0
                x_pop_f = 0
                a_f = 0
                x_pop = 0
                while j < len(a_list):
                    for keys in a_list[j]:
                        keys = str(keys)
                        x_pop = (a_list[j][keys]['population'])
                        if x_pop > x_pop_f:
                            a_f = a_list[j][keys]['geonameid']
                            x_pop_f = a_list[j][keys]['population']
                            x_lat_f = a_list[j][keys]['latitude']
                            x_long_f = a_list[j][keys]['longitude']
                            x_ctrycd_f = a_list[j][keys]['countrycode']
                    j = j + 1


        if (x_lat_f != []):
            df_lat.append(x_lat_f)
        if (x_long_f != []):
            df_long.append(x_long_f)
        if (x_ctrycd_f != []):
            df_ctrycd.append(x_ctrycd_f)

df['latitude'] = df_lat
df['longitude'] = df_long
df['countrycode'] = df_ctrycd
del df['countries']

df
                    
        

650
650


,headline,cities,latitude,longitude,countrycode
0,Zika Outbreak Hits Miami,Miami,25.7743,-80.1937,US
1,Could Zika Reach New York City,New York City,40.7143,-74.006,US
2,First Case of Zika in Miami Beach,Miami,25.7743,-80.1937,US
3,Mystery Virus Spreads in Recife Brazil,Recife,-8.05389,-34.8811,BR
4,Dallas man comes down with case of Zika,Dallas,32.7831,-96.8067,US
5,Trinidad confirms first Zika case,Trinidad,-14.8333,-64.9,BO
6,Zika Concerns are Spreading in Houston,Houston,29.7633,-95.3633,US
7,Geneve Scientists Battle to Find Cure,NaN,NaN,NaN,NaN
8,The CDC in Atlanta is Growing Worried,Atlanta,33.749,-84.388,US
9,Zika Infested Monkeys in Sao Paulo,NaN,NaN,NaN,NaN
